In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
/kaggle/input/sms-spam-collection-dataset/spam.csv


In [2]:
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer , ENGLISH_STOP_WORDS
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 

In [3]:
spam = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv', encoding = 'latin-1')
spam.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
spam = spam.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'], axis=1)
spam.columns = ["label", "text"]
spam.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
sw = stopwords.words('english')
def stopword(text):
    txt = [word.lower() for word in text.split() if word.lower() not in sw]
    return txt
spam['text'] = spam['text'].apply(stopword)
spam.head()

,label,text
0,ham,"[go, jurong, point,, crazy.., available, bugis..."
1,ham,"[ok, lar..., joking, wif, u, oni...]"
2,spam,"[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,ham,"[u, dun, say, early, hor..., u, c, already, sa..."
4,ham,"[nah, think, goes, usf,, lives, around, though]"


In [6]:
from nltk.stem.snowball import SnowballStemmer
s = SnowballStemmer('english')
def stemming(text):
    text = [s.stem(word) for word in text if word.split()]
    return ''.join(text)
spam['text'] = spam['text'].apply(stemming)

In [7]:
spam.head()

,label,text
0,ham,"gojurongpoint,crazy..availbugingreatworldlaebu..."
1,ham,oklar...jokewifuoni...
2,spam,freeentri2wklicompwinfacupfinaltkts21stmay2005...
3,ham,udunsayearlihor...ucalreadisay...
4,ham,"nahthinkgoeusf,livearoundthough"


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfid_vect = TfidfVectorizer()
tfid_matrix = tfid_vect.fit_transform(spam['text'])
print(f'Type : {type(tfid_matrix)}, Matrix at 0 : {tfid_matrix[0]}, shape : {tfid_matrix.shape}')

Type : <class 'scipy.sparse._csr.csr_matrix'>, Matrix at 0 :   (0, 1827)	0.5056391989470028
  (0, 1030)	0.5056391989470028
  (0, 2166)	0.48268727087494234
  (0, 3635)	0.5056391989470028, shape : (5572, 12124)


In [9]:
array = tfid_matrix.todense()

In [10]:
df = pd.DataFrame(array)
df[df[10] != 0].head()

,0,1,2,3,4,5,6,7,8,9,...,12114,12115,12116,12117,12118,12119,12120,12121,12122,12123
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5285,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
df['label'] = spam['label']

In [12]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,12115,12116,12117,12118,12119,12120,12121,12122,12123,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ham
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ham
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,spam
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ham
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ham


In [13]:
from sklearn.model_selection import train_test_split
features = df.drop('label', axis=1)
label = df['label']

x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=0.3)
print(f'X train shape : {x_train.shape}\nY train shape : {y_train.shape}\nX test shape : {x_test.shape}\nY test shape : {y_test.shape}')

X train shape : (3900, 12124)
Y train shape : (3900,)
X test shape : (1672, 12124)
Y test shape : (1672,)


In [14]:
from sklearn.pipeline import Pipeline

In [15]:
ber_pipe = Pipeline(steps = [('ber_model', BernoulliNB())])
multi_pipe = Pipeline(steps = [('multi_model', MultinomialNB())])
gauss_pipe = Pipeline(steps = [('gauss_model', GaussianNB())])

In [16]:
def model_evaluation(model):
    model.fit(x_train, y_train)
    y_pred_model = model.predict(x_test)
    acc_score = accuracy_score(y_test, y_pred_model)
    print(f'Accuracy Score of {model[0]} : {acc_score}')
    
model_evaluation(ber_pipe)
model_evaluation(multi_pipe)
model_evaluation(gauss_pipe)

Accuracy Score of BernoulliNB() : 0.8744019138755981
Accuracy Score of MultinomialNB() : 0.9001196172248804
Accuracy Score of GaussianNB() : 0.46710526315789475


**Multinomial NB is performing better than other models.**